### 🧩 O que são Runnables no LangChain?

No LangChain, Runnables são componentes reutilizáveis e encadeáveis que representam qualquer unidade executável de lógica — como um modelo, uma ferramenta, uma cadeia (chain) ou até mesmo uma função Python.

Eles fazem parte do novo sistema modular do LangChain, que permite compor fluxos complexos de forma simples e declarativa.


#### ✅ Para que servem?

Os Runnables permitem:

🔁 Encadear passos (chains) como entrada → processamento → saída.

🧪 Reutilizar lógica em diferentes contextos (como APIs, scripts ou agentes).

🔧 Compor pipelines com modelos, prompts, transformações, validações, etc.

⚡ Executar de forma assíncrona, paralela ou em lote (stream, batch, invoke).



In [ ]:
from langchain_core.runnables import RunnableLambda

# Cria um Runnable simples que transforma o texto para maiúsculas
uppercase = RunnableLambda(lambda x: x.upper())

result = uppercase.invoke("exemplo de texto")
print(result)  # Output: EXEMPLO DE TEXTO

#### 🚀 Métodos comuns dos Runnables

| Método     | Função                                              | 
| ---------- | --------------------------------------------------- | 
| `invoke()` | Executa uma chamada única                           | 
| `batch()`  | Executa em lote                                     | 
| `stream()` | Executa e retorna resultados parciais em tempo real | 


#### 🔗 Compondo Runnables
Você pode combinar vários Runnables em pipelines:

In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:

# Importa conectando com o cliente OpenAI
from helpers.llm import initialize_llm, logger, pretty_print

llm, _, _ = initialize_llm()

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Resuma o seguinte texto: {texto}")

# Encadeando os componentes
chain = prompt | llm

# Executando
response = chain.invoke({"texto": "LangChain é uma biblioteca para criar agentes com LLMs."})

pretty_print(response)

#### ✅ RunnableLambda

- RunnableLambda converte um python `callable` (pode-se entender aqui, a principio, como qualquer 'função' python) em um Runnable.
- Esse executável envolve o `callable` para que você possa utilizar ele em uma sequencia de componentes de LangChain
- Ele permite que funções customizadas participem de pipelines, agentes, LangGraph, e outras composições com LLMs.

In [ ]:
from langchain_core.runnables import RunnableLambda

# Etapas
strip_text = RunnableLambda(lambda x: x.strip())
capitalize_text = RunnableLambda(lambda x: x.capitalize())

print(strip_text.invoke("   langchain é poderoso   "))
print(capitalize_text.invoke("langchain é poderoso   "))

# Pipeline
#pipeline = strip_text | capitalize_text

#result = pipeline.invoke("   langchain é poderoso   ")
#print(result)  # Saída: Langchain é poderoso

langchain é poderoso
Langchain é poderoso   


##### 🧪 Exemplo com entrada em dicionário

Você também pode usar funções que processam dicionários — útil para quando seu fluxo carrega JSONs ou objetos estruturados:

In [11]:
format_input = RunnableLambda(lambda x: f"{x['nome']} tem {x['idade']} anos.")

result = format_input.invoke({"nome": "Ana", "idade": 30})
print(result)  # Saída: Ana tem 30 anos.

Ana tem 30 anos.


#### RunnableSequence

In [ ]:
# Veja o que é o RunnableLambda abaixo.
from langchain_core.runnables import RunnableLambda,RunnableSequence

# Função customizada
def adicionar(x: int) -> int:
    return x + 1


def somar(x: int) -> int:
    return x + 1

def multiplicar(x: int) -> int:
    return x * 2

runnable_1 = RunnableLambda(somar)
runnable_2 = RunnableLambda(multiplicar)


sequence = runnable_1 | runnable_2
# Or equivalently:
#sequence = RunnableSequence(first=runnable_1, last=runnable_2)
resposta = sequence.invoke(1)
print(resposta)

#### RunnableParallel

- Serve para executar componentes (ou operações) de forma paralela, quando ambos recebem o mesmo tipo de entrada. 
- Normalmente é representado por um dicionário usando chaves "{ ... }", desde que não seja inicio de Chain, neste caso precisamos declarar usando `RunnableParallel` literalmente ao invés de usar dicionários .

In [ ]:
from langchain_core.runnables import RunnableLambda,RunnableParallel

def dividir(x: int) -> int:
    return x / 2

runnable_1 = RunnableLambda(somar)
runnable_2 = RunnableLambda(multiplicar)
runnable_3 = RunnableLambda(dividir)

# sequence = runnable_1 | {  # o dicionário aqui é entendido como 'RunnableParallel'
#     "multiplicar": runnable_2,
#     "dividir": runnable_3,
# }
# Ou usando o equivalente:
# sequence = runnable_1 | RunnableParallel(
#     {"multiplicar": runnable_2, "dividir": runnable_3}
# )
# Ou também o equivalente:
sequence = runnable_1 | RunnableParallel(
    multiplicar=runnable_2,
    dividir=runnable_3,
)

resposta = sequence.invoke(2)
print(resposta)

#### RunnablePassthrough

- Executável para passar entradas inalteradas ou com chaves adicionais.
- Ao compor cadeias com várias etapas, às vezes você vai querer passar dados de etapas anteriores inalterados para uso como entrada para uma etapa posterior. A classe  `RunnablePassthrough` permite que você faça exatamente isso, e é tipicamente usada em conjunto com um `RunnableParallel` para passar dados para uma etapa posterior em suas cadeias construídas.

Exemplo 1: Sem transformar a entrada. 


In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough() | RunnablePassthrough() | RunnablePassthrough ()

# Independente de quantas vezes você "passar o resultado para frente", a entrada não é alterada.

resposta = chain.invoke("Olá")

print(resposta) # retorna Olá

Exemplo 2: Transformando a entrada.


In [ ]:
from langchain_core.runnables import RunnablePassthrough

def entrada_para_letras_maiusculas(entrada: str):
	saida = entrada.upper()
	return saida

chain = RunnablePassthrough() | RunnableLambda(entrada_para_letras_maiusculas) | RunnablePassthrough ()

# Neste caso vamos receber a entrada do usuário, passar para a função 'entrada_para_letras_maiusculas', transformar olá -> OLÁ e passar para frente.

resposta = chain.invoke("olá")

print(resposta) # retorna OLÁ

#### Operador Assign
- Normalmente é usado com a combinação `RunnablePassthrough.assign()` para que você possa criar uma saída em dicionário que será usada como entrada ao próximo componente com tal que você tenha a entrada inalterada e uma outra chave com a entrada transformada.

Exemplo:

In [ ]:
# https://python.langchain.com/docs/how_to/assign/

from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnablePassthrough() | RunnablePassthrough.assign(multiplica_3=lambda x: x["num"] * 3)

resposta = runnable.invoke({"num": 1})

print(resposta) # Saida {"num": 1, "multiplica_3": 3}


{'num': 1, 'multiplica_3': 3}


### Desafio com Runnables:

Processo:

1) Receber a entrada do tipo {"input": "SRP — Princípio da Responsabilidade Única."}  e passar para frente (testar uso do RunnablePassthrough)

2) Quando eu receber a entrada de (1) eu gostaria de criar um dicionário mantendo a entrada de (1) intacta, mas criando 
uma chave nova ("num_caract") tal que receba a entrada de (1) e conte o total de caracteres.

3) Usando a saída de (2) quero paralelizar a entrada em dois processos:
    1) primeiro, pegando a entrada textual e adicionando a frase " É um dos grandes principios do SOLID!" na chave chamada "transformar_entrada"
    2) segundo não farei nada, apenas passarei para frente a entrada sem qualquer alteração na chave "passa_para_frente".

4) Por fim, vou passar para frente a combinação do processo paralelo e imprimir o resultado.

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda, RunnableParallel

# Parte 1 - Recebe a entrada e passa para frente
parte_1_runnable = RunnablePassthrough()
 
# Parte 4 - RunnablePassthrough é um runnable que simplesmente passa a entrada para frente sem modificá-la.
parte_4_runnable = RunnablePassthrough()

# Juntando tudo:
chain = parte_1_runnable | parte_2_runnable | parte_3_runnable | parte_4_runnable

## Invocar:
resposta = chain.invoke({"input": "SRP — Princípio da Responsabilidade Única."})

print("------ RESPOSTA DO DESAFIO -----------------------")
print(resposta)
print(resposta["transformar_entrada"])
print(resposta["passa_para_frente"]) 
print("--------------------------------------------------")

------ RESPOSTA DO DESAFIO -----------------------
{'transformar_entrada': 'SRP — Princípio da Responsabilidade Única. É um dos grandes principios do SOLID!', 'passa_para_frente': {'input': 'SRP — Princípio da Responsabilidade Única.', 'num_caract': 42}}
SRP — Princípio da Responsabilidade Única. É um dos grandes principios do SOLID!
{'input': 'SRP — Princípio da Responsabilidade Única.', 'num_caract': 42}
--------------------------------------------------
